# CLASSIFICADOR DE REVIEWS

## DADOS 

In [ ]:
#https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/


In [50]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import random
from random import shuffle
from nltk.stem.snowball import SnowballStemmer
import string
import re
from nltk.corpus import stopwords


In [62]:
##carregando dados, filtrando-os e salvando e um novo objeto

dados = pd.read_csv('~/Documents/dsa/data/reviewFinal.csv',encoding='latin-1')
corpus = pd.DataFrame()
listTitle = []
listRecom = []
for i, r in dados.iterrows():
    if r['recommend'] != 'NI' and r['title'] != "" and r['title'] is not np.nan: # and r['site'] == "glassdoor":
        listTitle.append(r['title'])
        listRecom.append(r['recommend'])

## criando objeto para encoder
encoder = LabelEncoder()

In [52]:
corpus['title']     = listTitle
corpus['recommend'] = listRecom

In [53]:
### GERANDO LINHAS PARA DADOS DE TREINO E TESTE
x = [ i for i in range(len(corpus))]
shuffle(x)

rows_train =  x[0:round( len(corpus) * .75)] #25000
rows_test  = x[round( len(corpus) * .75):(len(corpus))]
dataset = []

In [54]:
### LIMPANDO TEXTO
stemmer = SnowballStemmer("english")
sClear = []
tr = str.maketrans("", "", string.punctuation)
i = 0
for i,text in corpus.iterrows():    
    #print(text['title'].split())
    clear = ""
    for word in text['title'].split():        
        word = word.lower() ## CONVERTENDO PARA MINUSCULO
        word = word.translate(tr) ##REMOVENDO PONTUAÇÕES        
        word = re.sub("\d"," ",word) ## REMOVENDO NUMEROS
        word = stemmer.stem(word) ## STEMM 
        
        clear = clear + ' ' + word
        clear = re.sub("\s+"," ",clear) ## REMOVENDO ESPAÇOS DUPLICADOS
        
        
    sClear.append(clear)
    text['title'] = clear
    i = i + 1
    

In [55]:
train   = corpus.iloc[rows_train, [0]]
train   = train['title'].tolist()
train_y = corpus.iloc[rows_train, [1] ] 
train_y = encoder.fit_transform(train_y)

test   = corpus.iloc[rows_test, [0]]
test   = test['title'].tolist()
test_y = corpus.iloc[rows_test, [1]]
test_y = encoder.fit_transform(test_y)

print(set(train_y), set(test_y) )

{0, 1} {0, 1}


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# MODELOS 

### NAIVE BAYES MULTINOMIAL

In [56]:
#criando countVectorizer
count_vect     = CountVectorizer(stop_words= stopwords.words('english'))
X_train_counts = count_vect.fit_transform(train)
count_vect.vocabulary_.get(u'algorithm')
X_train_counts.shape

(64565, 9094)

In [57]:
# criando TF-IDM
tfidf_transformer = TfidfTransformer()
X_train_tfidf     = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(64565, 9094)

In [58]:
#CRIANDO OBJETO NAIVE BAYES E TREINANDO O MODELO
clf = MultinomialNB().fit(X_train_tfidf, train_y)

In [59]:
# FAZENDO TRANSFORMAÇÃO NOS DADOS DE TESTE E TESTANDO O MODELO NB
X_new_counts = count_vect.transform(test)
X_new_tfidf  = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

In [60]:
## Verificando acuracia do modelo
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predicted) )

[[ 2890  4351]
 [  863 13418]]

 acuracia :  0.7577362698633956


### BINOMIAL NAIVE BAYES

In [28]:
nb = BernoulliNB()
nb.fit(X_train_tfidf, train_y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [29]:
predictBIN = nb.predict(X_new_tfidf)

In [30]:
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predictBIN) )

[[ 2940  4252]
 [  903 13391]]

 acuracia :  0.7555152192125105


### SVC

In [65]:
#Criando modelo SVC com Pipeline
modeloSVC  = Pipeline([('vect', TfidfVectorizer(stop_words=stopwords.words('english'))), 
                    ('clf', SVC(kernel = 'linear', probability = True))])

In [66]:
#treinando modelo
modeloSVC.fit(train, train_y)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [67]:
predictSVC = modeloSVC.predict(test)


In [68]:
print( confusion_matrix(test_y, predictSVC) ) 
print( '\n acuracia : ', accuracy_score(test_y, predictSVC) )


[[ 3157  4084]
 [ 1201 13080]]

 acuracia :  0.7544373199516774


### SVM

In [69]:
modeloSVM = Pipeline([('vect', CountVectorizer(stopwords.words('english'))),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=500, tol=None)),
])
modeloSVM.fit(train, train_y)  

predicted = modeloSVM.predict(test)
print( confusion_matrix(test_y, predicted) ) 
print( '\n acuracia : ', accuracy_score(test_y, predicted) )

[[  342  6899]
 [   52 14229]]

 acuracia :  0.6770281572344578


# FIM!!